# 問題36: 頻度上位10語

出現頻度が高い10語とその出現頻度をグラフで表示せよ．

In [ ]:
# 問題30で実装した関数を読み込む
import sys
sys.path.append('../..')  # 親ディレクトリをパスに追加
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 日本語表示のための設定
plt.rcParams['font.family'] = 'IPAexGothic'

# 形態素解析結果を読み込む関数
def load_mecab_result(file_path):
    """
    MeCabの解析結果ファイルを読み込み、各形態素を辞書のリストとして返す関数
    
    Args:
        file_path (str): MeCab出力ファイルのパス
        
    Returns:
        list: 文のリスト。各文は形態素（辞書）のリスト
    """
    sentences = []
    current_sentence = []
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # EOSは文の区切り
            if line == 'EOS\n':
                if current_sentence:
                    sentences.append(current_sentence)
                    current_sentence = []
                continue
                
            # 空行をスキップ
            if line == '\n':
                continue
                
            # タブで分割して表層形とそれ以外の情報に分ける
            try:
                surface, info = line.split('\t')
                
                # カンマで分割して品詞情報などを取得
                info_items = info.split(',')
                
                # 形態素情報を辞書として格納
                morpheme = {
                    'surface': surface,
                    'base': info_items[6],
                    'pos': info_items[0],
                    'pos1': info_items[1]
                }
                
                current_sentence.append(morpheme)
            except:
                # 不正な形式の行をスキップ
                continue
    
    # 最後の文が追加されていない場合に追加
    if current_sentence:
        sentences.append(current_sentence)
    
    return sentences

# 単語の出現頻度を計算する関数
def count_word_frequency(sentences):
    """
    形態素解析結果から単語の出現頻度を計算し、頻度順にソートする関数
    
    Args:
        sentences (list): 文のリスト。各文は形態素（辞書）のリスト
        
    Returns:
        list: (単語, 出現頻度)のタプルのリスト。出現頻度の降順でソート済み
    """
    # 全ての形態素の表層形を抽出
    words = [morpheme['surface'] for sentence in sentences for morpheme in sentence]
    
    # 単語の出現回数をカウント
    word_counts = Counter(words)
    
    # 出現頻度の降順でソート
    sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
    
    return sorted_word_counts

# 頻度上位n語をグラフで表示する関数
def plot_top_n_words(word_frequencies, n=10):
    """
    頻度上位n語とその出現頻度を棒グラフで表示する関数
    
    Args:
        word_frequencies (list): (単語, 出現頻度)のタプルのリスト
        n (int): 表示する単語の数
    """
    # 上位n語を抽出
    top_n = word_frequencies[:n]
    
    # 単語と頻度を分離
    words = [word for word, count in top_n]
    counts = [count for word, count in top_n]
    
    # データフレームに変換
    df = pd.DataFrame({'単語': words, '出現頻度': counts})
    
    # グラフの描画
    plt.figure(figsize=(12, 8), dpi=300)
    bars = plt.bar(df['単語'], df['出現頻度'], color=plt.cm.viridis(np.linspace(0, 1, n)))
    
    # グラフの装飾
    plt.title('出現頻度上位10語', fontsize=16)
    plt.xlabel('単語', fontsize=14)
    plt.ylabel('出現頻度', fontsize=14)
    plt.xticks(rotation=45, fontsize=12)
    plt.yticks(fontsize=12)
    
    # 各バーの上に数値を表示
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                 f'{height:.0f}',
                 ha='center', va='bottom', fontsize=12)
    
    plt.tight_layout()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    return plt.gcf()

In [ ]:
# 形態素解析結果の読み込み
file_path = '../../data/neko.txt.mecab'
sentences = load_mecab_result(file_path)

# 単語の出現頻度を計算
word_frequencies = count_word_frequency(sentences)

# 頻度上位10語をグラフで表示
fig = plot_top_n_words(word_frequencies, 10)
plt.show()

# 頻度上位10語を表形式でも表示
top10_df = pd.DataFrame(word_frequencies[:10], columns=['単語', '出現頻度'])
display(top10_df)

# 内容語（名詞、動詞、形容詞、副詞）のみの頻度上位10語
content_words = []
for sentence in sentences:
    for morpheme in sentence:
        if morpheme['pos'] in ['名詞', '動詞', '形容詞', '副詞']:
            content_words.append(morpheme['surface'])

content_word_counts = Counter(content_words)
sorted_content_word_counts = sorted(content_word_counts.items(), key=lambda x: x[1], reverse=True)

print("\n内容語（名詞、動詞、形容詞、副詞）の頻度上位10語:")
content_top10_df = pd.DataFrame(sorted_content_word_counts[:10], columns=['単語', '出現頻度'])
display(content_top10_df)

# 内容語の頻度上位10語をグラフで表示
fig2 = plot_top_n_words(sorted_content_word_counts, 10)
plt.title('内容語の出現頻度上位10語', fontsize=16)
plt.show()

## 解説

この問題では、出現頻度が高い10語とその出現頻度をグラフで表示しました。

### 実装のポイント

1. **頻度上位語の抽出**: 問題35で計算した単語の出現頻度から、上位10語を抽出しています。

2. **グラフの作成**: matplotlibを使用して、頻度上位10語を棒グラフで視覚化しています。

3. **グラフの装飾**: タイトル、軸ラベル、目盛りの回転、グリッド線などを設定して、グラフを見やすくしています。

4. **日本語の表示**: 日本語を正しく表示するために、フォントを設定しています。

5. **カラーパレット**: viridisカラーパレットを使用して、色覚異常の方にも識別しやすいグラフにしています。

### 結果の考察

出現頻度上位10語を見ると、日本語の文章で一般的に多用される助詞（「の」「に」「は」など）や助動詞が多く含まれています。これらは文法的な機能を担う単語であり、どのような文章でも高頻度で出現します。

内容語（名詞、動詞、形容詞、副詞）のみに絞った場合、小説『吾輩は猫である』の特徴を反映した単語が上位に来ています。例えば、「猫」「主人」などの単語は、この小説の主題や登場人物に関連しています。

### 機能語と内容語

単語は大きく以下の2種類に分けられます：

1. **機能語**: 助詞、助動詞、接続詞など、文法的な機能を担う単語。出現頻度が非常に高いが、テキストの内容を直接表すわけではない。

2. **内容語**: 名詞、動詞、形容詞、副詞など、テキストの内容を直接表す単語。テキストの主題や特徴を反映する。

テキスト分析では、目的に応じて機能語を除外し、内容語のみを対象とすることがあります。これにより、テキストの主題や特徴をより明確に把握することができます。